## Remap File Creation for mizuRoute for Gridded Runoff Field (CLM)

This section describes the steps to prepare a remap file for use with **mizuRoute**, converting runoff data from either structured or unstructured meshes to river network routing subbasins using **EASYMORE**.

### Overview

To generate the remap file, users should first call the **EASYMORE `nc_remapper`** function, providing:

- A **target shapefile** (e.g., river network subbasins), and  
- A **sample NetCDF file** containing the runoff field.

### Notes on Large River Networks

If the target river network topology is extensive or divided into multiple regions (e.g., continents or sub-regions), the remap files can be generated **separately for each part**, and later **merged into a single remap file**. After merging, the final **NetCDF remap file** can be created for use with mizuRoute.

### mizuRoute Control File

For the example provided here, the mizuRoute control file, including runoff file and remapping file specifications are as below:

```
! ****************************************************************************************************************************
! DEFINE RUNOFF FILE
! ----------------------------------
<fname_qsim>               CLM_sample.nc          ! name of netCDF containing the HRU runoff
<vname_qsim>               QRUNOFF                ! name of HRU runoff variable
<vname_time>               time                   ! name of time variable in the runoff file
<dname_time>               time                   ! name of time dimension
<dname_xlon>               lon                    ! name of x(j) dimension
<dname_ylat>               lat                    ! name of y(i) dimension
<units_qsim>               mm/s                   ! units of runoff
<dt_qsim>                  86400                  ! time interval of the runoff
! ****************************************************************************************************************************
! DEFINE RUNOFF MAPPING FILE
! ----------------------------------
<is_remap>                 T                      ! logical whether or not runnoff needs to be mapped to river network HRU
<fname_remap>              CLM_MedicineHat_mizuRoute_remapping.nc ! name of netCDF containing poly-poly mapping data
<dname_hru_remap>          polyid                 ! name of hru dimension name
<vname_hruid_in_remap>     RN_ID                  ! name of variable containing river network HRU
<vname_num_qhru>           RN_FR                  ! name of variable containing numbers of runoff HRUs within each river network HRU
<dname_data_remap>         intersect              ! name of data dimension name
<vname_weight>             weight                 ! name of variable contating areal weights of runoff HRUs within each river network HRU
<vname_i_index>            i_index                ! name of ylat index
<vname_j_index>            j_index                ! name of xlon index
! ****************************************************************************************************************************
```

In [ ]:
# loading EASYMORE and other packages
from easymore import Easymore
from mizuRoute_remapping_file_creation import mizuRoute_remapping_file_creation
import os

In [ ]:
# Generate easymore remapping file
esmr = Easymore()
# Set parameters individually
esmr.case_name            = "CLM_MedicineHat"
esmr.source_nc            = "../data/runoff/CLM/CLM_sample.nc"
esmr.var_names            = ["QRUNOFF"]
esmr.var_lon              = "lon"
esmr.var_lat              = "lat"
esmr.var_time             = "time"
esmr.target_shp           = "../data/subbasins/South_Saskatchewan_MedicineHat.shp"
esmr.target_shp_ID        = "COMID"
esmr.output_dir           = "../output/"
esmr.temp_dir             = "../temporary/"
esmr.only_create_remap_nc = True  # optional, only if you want remap file and not output data
# execute EASYMORE
esmr.nc_remapper()

In [ ]:
# Prepare mizuRoute remapping file from easymore remapping file and save
easymore_remapping_file = os.path.join(esmr.temp_dir, esmr.case_name+'_remapping.nc')
mizuRoute_remapping_file = mizuRoute_remapping_file_creation(easymore_remapping_file)
mizuRoute_remapping_file_name = os.path.join(esmr.output_dir, esmr.case_name+'_mizuRoute_remapping.nc')
if os.path.isfile(mizuRoute_remapping_file_name):
    os.remove(mizuRoute_remapping_file_name)
mizuRoute_remapping_file.to_netcdf(mizuRoute_remapping_file_name)
mizuRoute_remapping_file